In [88]:
import pandas as pd
import xarray as xr
import rioxarray as rio
import numpy as np
from pyhdf.SD import SD, SDC

# Estimate the rate of change in leaf biomass

## Introduction

## 1. Load data

In [112]:
biome_LAI_change = pd.read_excel('../data/supplementary_analysis/leaf_biomass_change/bg-18-4985-2021-t02.xlsx')
biome_LAI_change = biome_LAI_change.loc[[2,5,6,7,8,9,10,11]].set_index('Leaf area')
biome_LAI_change = biome_LAI_change.rename(index={'Anthro. vegetation':'Croplands'})

In [90]:
hdf = SD('../data/supplementary_analysis/leaf_biomass_change/MCD12C1.A2021001.061.2022217040006.hdf', SDC.READ)
data2D = hdf.select('Majority_Land_Cover_Type_1')
biome_map = xr.DataArray(data = data2D[:,:], coords={'y':np.linspace(90,-90,3600), 'x':np.linspace(-180,180,7200)}, dims=['y','x'])
biome_map.rio.write_crs(4326, inplace=True);

In [101]:
SLA_df = pd.read_csv('../data/supplementary_analysis/leaf_biomass_change/spat_1_pft_sla_large.csv')
SLA = xr.Dataset.from_dataframe(SLA_df.set_index(['lat','lon'])[['mean']])['mean']
LMA = 1000/SLA * 0.5 # convert m^2 kg (DW) to gC/m^2 
LMA.rio.write_crs(4236,inplace=True);
LMA = LMA.rename({'lat':'y','lon':'x'}).transpose('y', 'x')

In [92]:
biome_mapping = pd.read_csv('../data/supplementary_analysis/leaf_biomass_change/winkler_biome_mapping.csv',skiprows=1)


## 2. Run analysis

In [102]:

biome_map_rs = biome_map.rio.reproject_match(LMA)
biome_map_winkler = biome_map_rs.squeeze().copy()
biome_map_winkler[:] = biome_mapping['Biome'].values[biome_map_winkler]
biome_names = biome_mapping[['Biome','Biome name']].set_index('Biome').drop_duplicates()
LMA['biome'] = biome_map_winkler
biome_LMA_mean = LMA.groupby('biome').mean().drop_vars(['spatial_ref']).to_dataframe(name='LMA').merge(biome_names,left_index=True,right_index=True)

In [103]:
biome_LMA_mean

,LMA,Biome name
1,36.433162,Boreal Forests
2,62.335674,Tropical Forests
3,36.801805,Temperate Forests
4,50.465481,Shrublands
5,39.546450,Savannas
6,33.196343,Grasslands
7,35.423873,Croplands
8,33.820144,Others


In [122]:
merged_df = biome_LAI_change.merge(biome_LMA_mean.set_index('Biome name'),left_index=True,right_index=True,how='left')
merged_df.loc['Tundra','LMA'] = merged_df.loc['Grasslands','LMA']

In [123]:
leaf_biomass_change = (merged_df['Net leaf area change']*1e9*merged_df['LMA']/1e15)

In [131]:
leaf_and_herb_biomass_change.sort_values()

Tundra               0.000168
Boreal Forests       0.000643
Savannas             0.000895
Shrublands           0.001238
Tropical Forests     0.001503
Temperate Forests    0.001688
Croplands            0.002578
Grasslands           0.006156
dtype: object

In [127]:
(1+RS_ratio)

Savannas      1.642
Shrublands    2.887
Grasslands    5.224
Croplands     1.200
dtype: float64

In [124]:
# from https://www.nature.com/articles/s41597-020-0444-4/tables/6
# use 0.2 for croplands based on table 2 for maize and wheat
RS_ratio = pd.Series([0.642,1.887,4.224,0.2],index=['Savannas','Shrublands','Grasslands','Croplands'])
leaf_and_herb_biomass_change = leaf_biomass_change+(leaf_biomass_change*RS_ratio).fillna(0)

/tmp/ipykernel_2348834/376996825.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  leaf_and_herb_biomass_change = leaf_biomass_change+(leaf_biomass_change*RS_ratio).fillna(0)


In [125]:
leaf_and_herb_biomass_change.sum()


0.014869746552940767